In [80]:
path = "/content/drive/MyDrive"

#codes

In [81]:
cd $path

/content/drive/MyDrive


In [82]:
# try:
#     makeerror==1
# except:
#     !wget https://github.com/gravitational/teleconsole/releases/download/0.4.0/teleconsole-v0.4.0-linux-amd64.tar.gz
!tar -xvf teleconsole-v0.4.0-linux-amd64.tar.gz
# makeerror=False
import subprocess as sp
process = sp.Popen("/content/drive/MyDrive/teleconsole",shell=True,stdin=sp.PIPE,stdout=sp.PIPE,stderr=sp.PIPE)
for i in range(6):
    print(process.stdout.readline().decode())
!apt install screen
!screen -d -m bash

teleconsole
Starting local SSH server on localhost...

Requesting a disposable SSH proxy on teleconsole.com for root...

Checking status of the SSH tunnel...



Your Teleconsole ID: 91025eb0d6d62ab946df285da23274749bd00c07

WebUI for this session: https://teleconsole.com/s/91025eb0d6d62ab946df285da23274749bd00c07

Reading package lists... Done
Building dependency tree       
Reading state information... Done
screen is already the newest version (4.6.2-1ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [12]:
data_path = "/content/drive/MyDrive/workspace/PlaylistCompletion/spotifydata/testdata"

In [13]:
cd workspace/PlaylistCompletion/

/content/drive/My Drive/workspace/PlaylistCompletion


In [ ]:
import os

dirs = os.listdir(data_path)

for file in dirs:
  print(file)

In [7]:
import pandas as pd

In [8]:
import json

In [ ]:
files={}
for file in dirs:
  with open(f'spotifydata/testdata/{file}') as f:
    files[file] = json.load(f)



In [ ]:
files['mpd.slice.996000-996999.json']['playlists'][0]

{'collaborative': 'false',
 'duration_ms': 8515267,
 'modified_at': 1428710400,
 'name': 'worship music',
 'num_albums': 27,
 'num_artists': 20,
 'num_edits': 18,
 'num_followers': 1,
 'num_tracks': 31,
 'pid': 996000,
 'tracks': [{'album_name': 'Phil Wickham',
   'album_uri': 'spotify:album:12upYmvmRa3pmUBel7LiRr',
   'artist_name': 'Phil Wickham',
   'artist_uri': 'spotify:artist:5d1JhBfyb58upMXCZOdbQu',
   'duration_ms': 281880,
   'pos': 0,
   'track_name': 'Always Forever',
   'track_uri': 'spotify:track:2rb4cO7RczQFSvpjTJ4C2P'},
  {'album_name': 'The Eden Project',
   'album_uri': 'spotify:album:7j7D9b05MrsANLqbeDebcv',
   'artist_name': 'Chris Kipp',
   'artist_uri': 'spotify:artist:13RE7veV0Al5AwbLjR3uZ4',
   'duration_ms': 268440,
   'pos': 1,
   'track_name': 'Yearn',
   'track_uri': 'spotify:track:5bcCUdf8XlGEaDtzpI62EC'},
  {'album_name': 'Lead Me to the Cross',
   'album_uri': 'spotify:album:7AHlPi8rCQYCndkrdQN14E',
   'artist_name': 'Francesca Battistelli',
   'artist_uri

In [14]:
import datasets
import importlib
importlib.reload(datasets)

<module 'datasets' from '/content/drive/My Drive/workspace/PlaylistCompletion/datasets.py'>

In [15]:
d = datasets.Dataset('spotifydata/testdata')

In [16]:
%%capture
pip install -r requirements.txt

In [17]:
import inputs
importlib.reload(inputs)

<module 'inputs' from '/content/drive/My Drive/workspace/PlaylistCompletion/inputs.py'>

In [18]:
t2v, al2v, ar2v = inputs.word2vectors(d)

In [19]:
import numpy as np

In [57]:
import rnn
importlib.reload(rnn)

<module 'rnn' from '/content/drive/My Drive/workspace/PlaylistCompletion/rnn.py'>

In [58]:
train_gen = rnn.train_generator(t2v, al2v, ar2v, d)
rnn_model = rnn.rnn()

In [59]:
rnn_model.fit(train_gen, epochs=300)

Epoch 1/300


ValueError: ignored

In [40]:
model = inputs.cluster2rec(t2v, d, load=True)

In [47]:
ls = inputs.title2rec("Country chill", model, d)


/content/drive/My Drive/workspace/PlaylistCompletion/inputs.py:93: RuntimeWarning: invalid value encountered in float_scalars
  similar = []


In [ ]:
filter(lambda x: type)

In [49]:
ls.sort(key=lambda x: x[2], reverse=True)

In [55]:
'fly high'.strip()

'fly high'

In [50]:
ls[:300]

[(994019, 'fly ', nan),
 (994423, 'chill playlist', 0.9769992),
 (994407, 'Chillin.', 0.96391124),
 (994404, 'Chillax', 0.96157926),
 (994412, 'chill out ', 0.9569893),
 (994024, 'Playlist', 0.92620635),
 (995517, 'The Ultimate Playlist', 0.889528),
 (994398, 'Welcome Back', 0.8869601),
 (994368, 'quiet time ', 0.8769456),
 (995510, 'Christian Songs', 0.8556213),
 (995516, 'feelin it ', 0.853803),
 (995499, 'Jan 2015', 0.8512157),
 (995502, 'Awesome music', 0.84844667),
 (994436, 'One Dance', 0.8340094),
 (994022, 'Hip Hop', 0.82327634),
 (994025, 'oth', nan),
 (994100, 'Chill Mix', 0.9226472),
 (994098, 'The Playlist', 0.8813677),
 (994036, 'Mixed Playlist', 0.88111806),
 (994180, 'Just Listen', 0.87194073),
 (994107, 'Hiphop', 0.8711134),
 (994088, 'Tasty Jams', 0.8610036),
 (994017, 'New', 0.86018753),
 (994744, 'feels', 0.8484456),
 (994089, 'Good', 0.82435983),
 (995526, 'The Classics', 0.8207369),
 (995500, 'Morning', 0.8200146),
 (995540, 'Thinking ', 0.81909335),
 (995542, 'Jun

In [31]:
inputs.playlist2vec

{}

In [4]:
f = open('text.txt', 'r')
f.read()
f.close()

In [ ]:
gen = d.playlist_gen()

In [ ]:
next(gen)

In [ ]:
len(d.albums_uri2id.values())

40826

#hello


In [ ]:
cat datasets.py

In [ ]:
def count():
  i = 0
  while i < 10:
    yield i
    i += 2
c = count()

In [ ]:
for i in c:
  print(i)

0
2
4
6
8


In [13]:
!echo "fasttext" >> requirements.txt

In [ ]:
ls

In [9]:
pip install -r requirements.py

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.py'


In [16]:
pip install fasttext

     |████████████████████████████████| 71kB 4.6MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3097844 sha256=a9918ca612c0cc41669d508f7f0b546c3b6f959f844142aa739f89b533936b3f
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [17]:
import fasttext

In [20]:
fastmodel = fasttext.train_unsupervised('text.txt', model='skipgram', lr=0.1, epoch=5, loss='softmax', ws=5)

In [5]:
import numpy as np



In [6]:
np.concatenate((np.array([1,2]), np.array([3,4])), axis=None)

array([1, 2, 3, 4])